In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [3]:
filename = 'cleaned_fraudTrain.csv'

In [4]:
# import Dataset
path = '../dataset/midterm_dataset/'+filename
df = pd.read_csv(path)

row_num = df.shape[0]
feature_num = df.shape[1]

print(df.shape)
df.head()

(1296675, 14)


,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,misc_net,4.97,0,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,0,"28654, Moravian Falls, NC",31,Late Night,2019,1
1,grocery_pos,107.23,0,48.8878,-118.2105,149,Special educational needs teacher,1325376044,0,"99160, Orient, WA",41,Late Night,2019,1
2,entertainment,220.11,1,42.1808,-112.2620,4154,Nature conservation officer,1325376051,0,"83252, Malad City, ID",57,Late Night,2019,1
3,gas_transport,45.00,1,46.2306,-112.1138,1939,Patent attorney,1325376076,0,"59632, Boulder, MT",52,Late Night,2019,1
4,misc_pos,41.96,1,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,0,"24433, Doe Hill, VA",33,Late Night,2019,1


In [5]:
label = 'is_fraud'
label

'is_fraud'

In [6]:
df.groupby(['job', label]).size()

job                              is_fraud
Academic librarian               0           1030
                                 1             11
Accountant, chartered            1             11
Accountant, chartered certified  0            525
                                 1              9
                                             ... 
Web designer                     1             18
Wellsite geologist               0           2581
                                 1             20
Writer                           0            489
                                 1             15
Length: 918, dtype: int64

In [7]:
features = df.loc[:, df.columns != label].columns.tolist()
print(features)

['category', 'amt', 'is_male', 'lat', 'long', 'city_pop', 'job', 'unix_time', 'state_city', 'age', 'trans_time', 'trans_year', 'trans_month']


In [8]:
X = df[features]
X.head()

,category,amt,is_male,lat,long,city_pop,job,unix_time,state_city,age,trans_time,trans_year,trans_month
0,misc_net,4.97,0,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,"28654, Moravian Falls, NC",31,Late Night,2019,1
1,grocery_pos,107.23,0,48.8878,-118.2105,149,Special educational needs teacher,1325376044,"99160, Orient, WA",41,Late Night,2019,1
2,entertainment,220.11,1,42.1808,-112.2620,4154,Nature conservation officer,1325376051,"83252, Malad City, ID",57,Late Night,2019,1
3,gas_transport,45.00,1,46.2306,-112.1138,1939,Patent attorney,1325376076,"59632, Boulder, MT",52,Late Night,2019,1
4,misc_pos,41.96,1,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,"24433, Doe Hill, VA",33,Late Night,2019,1


In [9]:
y = df[label]
y.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [10]:
def find_prob_each(label, feature, df=df):
    """
    helper function for group_entropy function
    """

    temp_df = df.groupby([feature, label])
    prob_list = []
    if len(temp_df.size())< 2000 and feature not in ['age', 'lat', 'long', 'city_pop']:  
        for r in range(len(temp_df.size())):
                feature_prob = temp_df.size().values[r]/ \
                    len(df[feature][df[feature] == temp_df.size().index[r][0]])
                prob_feature = {}
                prob_feature[temp_df.size().index[r]] = feature_prob
                featu = ''
                if len(prob_list) == 0:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                elif temp_df.size().index[r][0] in prob_list[-1]: #append 2nd value
#                     print(df.groupby([feature, label]).size().index[r][0], r,feature_prob, temp_df.size().index[r][1])
#                     print(prob_list[-1][temp_df.size().index[r][0]])
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][temp_df.size().index[r][0]][1] = feature_prob
                    else:
                        prob_list[-1][temp_df.size().index[r][0]][0] = feature_prob
                else:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                    
    else: #numeric feature
        greater_than_eq_df = df[[label, feature]][df[feature] >=df[feature].mean()]
        less_than_df = df[[label, feature]][df[feature] < df[feature].mean()]
        condition_dfs = [greater_than_eq_df, less_than_df]
    
        prob_list = [{'greater_than_eq_mean': []}, {'less_than_mean': []}]
    
        for d in condition_dfs:
            for r in range(len(d[label].value_counts())):
                feature_prob = d[label].value_counts()[r]/len(d)
#                 print(feature_prob)
                if  d.equals(greater_than_eq_df):
                    prob_list[0]['greater_than_eq_mean'].append(feature_prob)

                else:
                    prob_list[1]['less_than_mean'].append(feature_prob)
    p_dict = {}
    [p_dict.update(i) for i in prob_list]
    
    return p_dict

In [11]:
find_prob_each(label, 'category')

{'entertainment': [0.9975216457123407, 0.0024783542876592847],
 'food_dining': [0.998349023080876, 0.0016509769191239982],
 'gas_transport': [0.9953060557956539, 0.004693944204346076],
 'grocery_net': [0.9970518349027545, 0.0029481650972454456],
 'grocery_pos': [0.985902392468335, 0.014097607531665023],
 'health_fitness': [0.9984513094004355, 0.0015486905995645036],
 'home': [0.9983917475531008, 0.0016082524468992406],
 'kids_pets': [0.9978856106515681, 0.0021143893484319018],
 'misc_net': [0.985542054450361, 0.014457945549638947],
 'misc_pos': [0.9968614650681062, 0.003138534931893792],
 'personal_care': [0.9975759712642411, 0.002424028735758831],
 'shopping_net': [0.9824385142962591, 0.017561485703740914],
 'shopping_pos': [0.9927746160175535, 0.007225383982446517],
 'travel': [0.9971362974300738, 0.0028637025699261858]}

In [12]:
import math
def group_entropy(feature, df=df):
    """
    give most pure feature in sorted_feature part
    note: the most pure value = 0
    """
    an_entropy = {}
    prob_dict = find_prob_each(label, feature, df)
    
    for key in prob_dict:
#         key = list(prob.keys())[0]
#         print(key)
        index0 = prob_dict[key][0]*np.log2(prob_dict[key][0])
        index1 = prob_dict[key][1]*np.log2(prob_dict[key][1])
        if math.isnan(index0): index0 = 0
        if math.isnan(index1): index1 = 0
        an_entropy[key] = -index0 -index1

    output = {
        feature: an_entropy,
        'sorted_feature' : []
    }
    t = [(k, v) for k, v in output[feature].items()]
    t.sort(key=lambda x : x[1])
    output['sorted_feature'] = t

    return output

In [13]:
prob_dict = find_prob_each(label, 'category')
for i in prob_dict:
    print(prob_dict[i])

[0.9975216457123407, 0.0024783542876592847]
[0.998349023080876, 0.0016509769191239982]
[0.9953060557956539, 0.004693944204346076]
[0.9970518349027545, 0.0029481650972454456]
[0.985902392468335, 0.014097607531665023]
[0.9984513094004355, 0.0015486905995645036]
[0.9983917475531008, 0.0016082524468992406]
[0.9978856106515681, 0.0021143893484319018]
[0.985542054450361, 0.014457945549638947]
[0.9968614650681062, 0.003138534931893792]
[0.9975759712642411, 0.002424028735758831]
[0.9824385142962591, 0.017561485703740914]
[0.9927746160175535, 0.007225383982446517]
[0.9971362974300738, 0.0028637025699261858]


In [14]:
group_entropy('category')

{'category': {'entertainment': 0.025024705714635345,
  'food_dining': 0.017638984226251205,
  'gas_transport': 0.04306359296929188,
  'grocery_net': 0.02902920569589835,
  'grocery_pos': 0.10687232007048204,
  'health_fitness': 0.01668917410800163,
  'home': 0.01724340082592481,
  'kids_pets': 0.021834689809823667,
  'misc_net': 0.10907376058187168,
  'misc_pos': 0.030619928795145204,
  'personal_care': 0.02455376891667575,
  'shopping_net': 0.12752085954948783,
  'shopping_pos': 0.06177833690432475,
  'travel': 0.028317808679317737},
 'sorted_feature': [('health_fitness', 0.01668917410800163),
  ('home', 0.01724340082592481),
  ('food_dining', 0.017638984226251205),
  ('kids_pets', 0.021834689809823667),
  ('personal_care', 0.02455376891667575),
  ('entertainment', 0.025024705714635345),
  ('travel', 0.028317808679317737),
  ('grocery_net', 0.02902920569589835),
  ('misc_pos', 0.030619928795145204),
  ('gas_transport', 0.04306359296929188),
  ('shopping_pos', 0.06177833690432475),
  (

In [15]:
find_prob_each(label, 'trans_time')

{'Early Morning': [0.9987882656830931, 0.0012117343169069448],
 'Eve': [0.9988477987997268, 0.001152201200273171],
 'Late Night': [0.9879008177410784, 0.01209918225892156],
 'Morning': [0.9989780755494148, 0.0010219244505852603],
 'Night': [0.9804176957103282, 0.019582304289671826],
 'Noon': [0.998771602198105, 0.0012283978018950237]}

In [16]:
find_prob_each(label, 'trans_time', df.iloc[:10,])

{'Late Night': [1.0, 0.0]}

In [17]:
df.iloc[:10,]['trans_time'].value_counts()

Late Night    10
Name: trans_time, dtype: int64

In [18]:
df.iloc[:10,]

,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,misc_net,4.97,0,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,0,"28654, Moravian Falls, NC",31,Late Night,2019,1
1,grocery_pos,107.23,0,48.8878,-118.2105,149,Special educational needs teacher,1325376044,0,"99160, Orient, WA",41,Late Night,2019,1
2,entertainment,220.11,1,42.1808,-112.2620,4154,Nature conservation officer,1325376051,0,"83252, Malad City, ID",57,Late Night,2019,1
3,gas_transport,45.00,1,46.2306,-112.1138,1939,Patent attorney,1325376076,0,"59632, Boulder, MT",52,Late Night,2019,1
4,misc_pos,41.96,1,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,0,"24433, Doe Hill, VA",33,Late Night,2019,1
5,gas_transport,94.63,0,40.3750,-75.2045,2158,Transport planner,1325376248,0,"18917, Dublin, PA",58,Late Night,2019,1
6,grocery_net,44.54,0,37.9931,-100.9893,2691,Arboriculturist,1325376282,0,"67851, Holcomb, KS",26,Late Night,2019,1
7,gas_transport,71.65,1,38.8432,-78.6003,6018,"Designer, multimedia",1325376308,0,"22824, Edinburg, VA",72,Late Night,2019,1
8,misc_pos,4.27,0,40.3359,-79.6607,1472,Public affairs consultant,1325376318,0,"15665, Manor, PA",78,Late Night,2019,1
9,grocery_pos,198.39,0,36.5220,-87.3490,151785,Pathologist,1325376361,0,"37040, Clarksville, TN",45,Late Night,2019,1


In [19]:
find_prob_each(label, 'unix_time')

{'greater_than_eq_mean': [0.9943265233107567, 0.005673476689243372],
 'less_than_mean': [0.9940961310914984, 0.005903868908501602]}

In [20]:
df[['category', label]][df['category'] == 'misc_net'].value_counts()

category  is_fraud
misc_net  0           62372
          1             915
dtype: int64

In [21]:
'misc_ne' in find_prob_each(label, 'category')

False

In [22]:
find_prob_each(label, 'category')['misc_net'] #0 1

[0.985542054450361, 0.014457945549638947]

In [23]:
find_prob_each(label, 'amt')

{'greater_than_eq_mean': [0.9864942104874991, 0.013505789512500923],
 'less_than_mean': [0.9980922290206041, 0.0019077709793958416]}

In [37]:
def extract_point_feature(feature, value, df=df):
    #hv to find mean later and predict which
    not_fraud = []
    fraud = []
    #check type
    if type(value) != str and feature not in ['is_male', 'trans_year', 'trans_month']:
        mean_val = df[feature].mean()
        if value >= mean_val:
            value = 'greater_than_eq_mean'
        else:
            value = 'less_than_mean'
        print(value)
    prob_dict = find_prob_each(label, feature)
    
    #append value
    if value not in prob_dict:
        not_fraud.append(0)
        fraud.append(0)
        
        print(not_fraud,fraud)
        
        return [0, 0]
    elif value in prob_dict:
        not_fraud.append(100*prob_dict[value][0])
        fraud.append(100*prob_dict[value][1])
                
        print(not_fraud,fraud)
        
        return [100*prob_dict[value][0], 100*prob_dict[value][1]]

In [39]:
def add_point(an_df_row, features=[], df=df):
    an_row = an_df_row.values.tolist()[0]
    for i, feature in enumerate(features):
        extract_point_feature(feature, an_row[i], df)
        print(feature,'done')
        

In [26]:
features

['category',
 'amt',
 'is_male',
 'lat',
 'long',
 'city_pop',
 'job',
 'unix_time',
 'state_city',
 'age',
 'trans_time',
 'trans_year',
 'trans_month']

In [28]:
["foo", "bar", "baz"].index("bar")

1

In [27]:
X.values.tolist()

[['misc_net',
  4.97,
  0,
  36.0788,
  -81.1781,
  3495,
  'Psychologist, counselling',
  1325376018,
  '28654, Moravian Falls, NC',
  31,
  'Late Night',
  2019,
  1],
 ['grocery_pos',
  107.23,
  0,
  48.8878,
  -118.2105,
  149,
  'Special educational needs teacher',
  1325376044,
  '99160, Orient, WA',
  41,
  'Late Night',
  2019,
  1],
 ['entertainment',
  220.11,
  1,
  42.1808,
  -112.262,
  4154,
  'Nature conservation officer',
  1325376051,
  '83252, Malad City, ID',
  57,
  'Late Night',
  2019,
  1],
 ['gas_transport',
  45.0,
  1,
  46.2306,
  -112.1138,
  1939,
  'Patent attorney',
  1325376076,
  '59632, Boulder, MT',
  52,
  'Late Night',
  2019,
  1],
 ['misc_pos',
  41.96,
  1,
  38.4207,
  -79.4629,
  99,
  'Dance movement psychotherapist',
  1325376186,
  '24433, Doe Hill, VA',
  33,
  'Late Night',
  2019,
  1],
 ['gas_transport',
  94.63,
  0,
  40.375,
  -75.2045,
  2158,
  'Transport planner',
  1325376248,
  '18917, Dublin, PA',
  58,
  'Late Night',
  2019,


In [ ]:
extract_point_feature('category', test_row['category'])

In [40]:
add_point(test_row, features)

[98.5542054450361] [1.4457945549638946]
category done
less_than_mean
[99.80922290206041] [0.19077709793958417]
amt done
[99.47384213573605] [0.5261578642639495]
is_male done
less_than_mean
[99.43461818312478] [0.5653818168752189]
lat done
greater_than_eq_mean
[99.39216773621463] [0.6078322637853727]
long done
less_than_mean
[99.43123656406318] [0.5687634359368186]
city_pop done
[99.83074753173484] [0.16925246826516221]
job done
less_than_mean
[99.40961310914984] [0.5903868908501602]
unix_time done
[100.0] [0.0]
state_city done
less_than_mean
[99.51208092047732] [0.48791907952268493]
age done
[98.79008177410785] [1.209918225892156]
trans_time done
[99.43558414878089] [0.5644158512191166]
trans_year done
[99.18932080552293] [0.8106791944770689]
trans_month done


In [ ]:
type('sd') == str

In [32]:
test_row = X.iloc[:1,]
k = test_row['amt']

In [33]:
test_row

,category,amt,is_male,lat,long,city_pop,job,unix_time,state_city,age,trans_time,trans_year,trans_month
0,misc_net,4.97,0,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,"28654, Moravian Falls, NC",31,Late Night,2019,1


In [34]:
test_row.values.tolist()

[['misc_net',
  4.97,
  0,
  36.0788,
  -81.1781,
  3495,
  'Psychologist, counselling',
  1325376018,
  '28654, Moravian Falls, NC',
  31,
  'Late Night',
  2019,
  1]]